### this compares two signals using a perm test and plots their wavelets

In [ ]:
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

import mne.time_frequency
import mne
from ieeg.io import get_data, raw_from_layout
from ieeg.navigate import trial_ieeg, channel_outlier_marker, crop_empty_data, outliers_to_nan
from ieeg.calc.scaling import rescale
from ieeg.calc.fast import mean_diff
from ieeg.calc.stats import time_perm_cluster
import os
from ieeg.timefreq.utils import wavelet_scaleogram, crop_pad
from ieeg.viz.ensemble import chan_grid

import numpy as np
from utils import calculate_RTs, get_good_data
from wavelet_functions import make_and_get_sig_wavelet_differences, load_and_get_sig_wavelet_differences

['C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\Pythonwin']


In [2]:
HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data('GlobalLocal', root=LAB_root)

# subjects = ['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103', 'D0107A', 'D0110']
subjects = ['D0057']

# this is a toggle for which version to run - the one that makes the wavelets in this notebook directly, or the one that loads them
make_wavelets=False

maybe consider using this big ass dict from roi_analysis.ipynb to map conditions onto events

In [ ]:
# # # example of how to use this with multiple conditions, even matching any value in a list. Although I only ever have two conditions of a type so not super necessary.
# # # make sure to use the correct column names and values that match with what combinedData uses.

# # congruency
# stimulus_congruency_conditions = {
#     "Stimulus_c": {
#         "BIDS_events": ["Stimulus/c25.0", "Stimulus/c75.0"],
#         "congruency": "c"
#     },
#     "Stimulus_i": {
#         "BIDS_events": ["Stimulus/i25.0", "Stimulus/i75.0"],
#         "congruency": "i"
#     }
# }

# # switch type
# stimulus_switch_type_conditions = {
#     "Stimulus_r": {
#         "BIDS_events": ["Stimulus/r25.0", "Stimulus/r75.0"],
#         "switchType": "r"
#     },
#     "Stimulus_s": {        
#         "BIDS_events": ["Stimulus/s25.0", "Stimulus/s75.0"],
#         "switchType": "s"
#     }
# }

# # #  ir vs is
# # output_names = ["Stimulus_ir_fixationCrossBase_1sec_mirror", "Stimulus_is_fixationCrossBase_1sec_mirror"]
# # output_names_conditions = {
# #     "Stimulus_ir_fixationCrossBase_1sec_mirror": {
# #         "congruency": "i",
# #         "switchType": "r"
# #     },
# #     "Stimulus_is_fixationCrossBase_1sec_mirror": {
# #         "congruency": "i",
# #         "switchType": "s"
# #     }
# # }

# # #  cr vs cs
# # output_names = ["Stimulus_cr_fixationCrossBase_1sec_mirror", "Stimulus_cs_fixationCrossBase_1sec_mirror"]
# # output_names_conditions = {
# #     "Stimulus_cr_fixationCrossBase_1sec_mirror": {
# #         "congruency": "c",
# #         "switchType": "r"
# #     },
# #     "Stimulus_cs_fixationCrossBase_1sec_mirror": {
# #         "congruency": "c",
# #         "switchType": "s"
# #     }
# # }

# # #  is vs cs
# # output_names = ["Stimulus_cs_fixationCrossBase_1sec_mirror", "Stimulus_is_fixationCrossBase_1sec_mirror"]
# # output_names_conditions = {
# #     "Stimulus_cs_fixationCrossBase_1sec_mirror": {
# #         "congruency": "c",
# #         "switchType": "s"
# #     },
# #     "Stimulus_is_fixationCrossBase_1sec_mirror": {
# #         "congruency": "i",
# #         "switchType": "s"
# #     }
# # }

# # #  ir vs cr
# # output_names = ["Stimulus_cr_fixationCrossBase_1sec_mirror", "Stimulus_ir_fixationCrossBase_1sec_mirror"]
# # output_names_conditions = {
# #     "Stimulus_cr_fixationCrossBase_1sec_mirror": {
# #         "congruency": "c",
# #         "switchType": "r"
# #     },
# #     "Stimulus_ir_fixationCrossBase_1sec_mirror": {
# #         "congruency": "i",
# #         "switchType": "r"
# #     }
# # }

# # # # main effect interaction effects (run this with the anova code. Ugh make everything more modular later.)

# stimulus_main_effect_conditions = {
#     "Stimulus_ir": {
#         "BIDS_events": ["Stimulus/i25.0/r25.0", "Stimulus/i25.0/r75.0", "Stimulus/i75.0/r25.0", "Stimulus/i75.0/r75.0"],
#         "congruency": "i",
#         "switchType": "r"
#     },
#     "Stimulus_is": {
#         "BIDS_events": ["Stimulus/i25.0/s25.0", "Stimulus/i25.0/s75.0", "Stimulus/i75.0/s25.0", "Stimulus/i75.0/s75.0"],
#         "congruency": "i",
#         "switchType": "s"
#     },
#     "Stimulus_cr": {
#         "BIDS_events": ["Stimulus/c25.0/r25.0", "Stimulus/c25.0/r75.0", "Stimulus/c75.0/r25.0", "Stimulus/c75.0/r75.0"],
#         "congruency": "c",
#         "switchType": "r"
#     },
#     "Stimulus_cs": {
#         "BIDS_events": ["Stimulus/c25.0/s25.0", "Stimulus/c25.0/s75.0", "Stimulus/c75.0/s25.0", "Stimulus/c75.0/s75.0"],
#         "congruency": "c",
#         "switchType": "s"
#     }
# }

# # # block interaction contrasts for lwpc

# stimulus_lwpc_conditions = {
#     "Stimulus_c25": {
#         "BIDS_events": "Stimulus/c25.0",
#         "congruency": "c",
#         "congruencyProportion": "75%" #this is flipped because the BIDS events are saved in terms of incongruency proportion
#     },
#     "Stimulus_c75": {
#         "BIDS_events": "Stimulus/c75.0",
#         "congruency": "c",
#         "congruencyProportion": "25%" #this is flipped because the BIDS events are saved in terms of incongruency proportion
#     },
#     "Stimulus_i25": {
#         "BIDS_events": "Stimulus/i25.0",
#         "congruency": "i",
#         "congruencyProportion": "75%" #this is flipped because the BIDS events are saved in terms of incongruency proportion
#     },
#     "Stimulus_i75": {
#         "BIDS_events": "Stimulus/i75.0",
#         "congruency": "i",
#         "congruencyProportion": "25%" #this is flipped because the BIDS events are saved in terms of incongruency proportion
#     }
# }

# # # block interaction contrasts for lwps

# stimulus_lwps_conditions = {
#     "Stimulus_s25": {
#         "BIDS_events": "Stimulus/s25.0",
#         "switchType": "s",
#         "switchProportion": "25%"
#     },
#     "Stimulus_s75": {
#         "BIDS_events": "Stimulus/s75.0",
#         "switchType": "s",
#         "switchProportion": "75%"
#     },
#     "Stimulus_r25": {
#         "BIDS_events": "Stimulus/r25.0",
#         "switchType": "r",
#         "switchProportion": "25%"
#     },
#     "Stimulus_r75": {
#         "BIDS_events": "Stimulus/r75.0",
#         "switchType": "r",
#         "switchProportion": "75%"
#     }
# }

# # all 16 trial types
# stimulus_experiment_conditions = {
#     "Stimulus_i25s25": {
#         "BIDS_events": "Stimulus/i25.0/s25.0",
#         "congruency": "i",
#         "congruencyProportion": "75%",
#         "switchType": "s",
#         "switchProportion": "25%"
#     },
#     "Stimulus_i25s75": {
#         "BIDS_events": "Stimulus/i25.0/s75.0",
#         "congruency": "i",
#         "congruencyProportion": "75%",
#         "switchType": "s",
#         "switchProportion": "75%"
#     },
#     "Stimulus_i75s25": {
#         "BIDS_events": "Stimulus/i75.0/s25.0",
#         "congruency": "i",
#         "congruencyProportion": "25%",
#         "switchType": "s",
#         "switchProportion": "25%"
#     },
#     "Stimulus_i75s75": {
#         "BIDS_events": "Stimulus/i75.0/s75.0",
#         "congruency": "i",
#         "congruencyProportion": "25%",
#         "switchType": "s",
#         "switchProportion": "75%"
#     },
#     "Stimulus_i25r25": {
#         "BIDS_events": "Stimulus/i25.0/r25.0",
#         "congruency": "i",
#         "congruencyProportion": "75%",
#         "switchType": "r",
#         "switchProportion": "25%"
#     },
#     "Stimulus_i25r75": {
#         "BIDS_events": "Stimulus/i25.0/r75.0",
#         "congruency": "i",
#         "congruencyProportion": "75%",
#         "switchType": "r",
#         "switchProportion": "75%"
#     },
#     "Stimulus_i75r25": {
#         "BIDS_events": "Stimulus/i75.0/r25.0",
#         "congruency": "i",
#         "congruencyProportion": "25%",
#         "switchType": "r",
#         "switchProportion": "25%"
#     },
#     "Stimulus_i75r75": {
#         "BIDS_events": "Stimulus/i75.0/r75.0",
#         "congruency": "i",
#         "congruencyProportion": "25%",
#         "switchType": "r",
#         "switchProportion": "75%"
#     },
#     "Stimulus_c25s25": {
#         "BIDS_events": "Stimulus/c25.0/s25.0",
#         "congruency": "c",
#         "congruencyProportion": "75%",
#         "switchType": "s",
#         "switchProportion": "25%"
#     },
#     "Stimulus_c25s75": {
#         "BIDS_events": "Stimulus/c25.0/s75.0",
#         "congruency": "c",
#         "congruencyProportion": "75%",
#         "switchType": "s",
#         "switchProportion": "75%"
#     },
#     "Stimulus_c75s25": {
#         "BIDS_events": "Stimulus/c75.0/s25.0",
#         "congruency": "c",
#         "congruencyProportion": "25%",
#         "switchType": "s",
#         "switchProportion": "25%"
#     },
#     "Stimulus_c75s75": {
#         "BIDS_events": "Stimulus/c75.0/s75.0",
#         "congruency": "c",
#         "congruencyProportion": "25%",
#         "switchType": "s",
#         "switchProportion": "75%"
#     },
#     "Stimulus_c25r25": {
#         "BIDS_events": "Stimulus/c25.0/r25.0",
#         "congruency": "c",
#         "congruencyProportion": "75%",
#         "switchType": "r",
#         "switchProportion": "25%"
#     },
#     "Stimulus_c25r75": {
#         "BIDS_events": "Stimulus/c25.0/r75.0",
#         "congruency": "c",
#         "congruencyProportion": "75%",
#         "switchType": "r",
#         "switchProportion": "75%"
#     },
#     "Stimulus_c75r25": {
#         "BIDS_events": "Stimulus/c75.0/r25.0",
#         "congruency": "c",
#         "congruencyProportion": "25%",
#         "switchType": "r",
#         "switchProportion": "25%"
#     },
#     "Stimulus_c75r75": {
#         "BIDS_events": "Stimulus/c75.0/r75.0",
#         "congruency": "c",
#         "congruencyProportion": "25%",
#         "switchType": "r",
#         "switchProportion": "75%"
#     }
# }

# # stimulus details
# stimulus_conditions = {
#     "Stimulus_bigSsmallHtaskG": {
#         "BIDS_events": "Stimulus/BigLetters/SmallLetterh/Taskg",
#         "bigLetter": "s",
#         "smallLetter": "h",
#         "task": "g"
#     },
#     "Stimulus_bigSsmallHtaskL": {
#         "BIDS_events": "Stimulus/BigLetters/SmallLetterh/Taskl",
#         "bigLetter": "s",
#         "smallLetter": "h",
#         "task": "l"
#     },
#     "Stimulus_bigSsmallStaskG": {
#         "BIDS_events": "Stimulus/BigLetters/SmallLetters/Taskg",
#         "bigLetter": "s",
#         "smallLetter": "s",
#         "task": "g"
#     },
#     "Stimulus_bigSsmallStaskL": {
#         "BIDS_events": "Stimulus/BigLetters/SmallLetters/Taskl",
#         "bigLetter": "s",
#         "smallLetter": "s",
#         "task": "l"
#     },
#     "Stimulus_bigHsmallHtaskG": {
#         "BIDS_events": "Stimulus/BigLetterh/SmallLetterh/Taskg",
#         "bigLetter": "h",
#         "smallLetter": "h",
#         "task": "g"
#     },
#     "Stimulus_bigHsmallHtaskL": {
#         "BIDS_events": "Stimulus/BigLetterh/SmallLetterh/Taskl",
#         "bigLetter": "h",
#         "smallLetter": "h",
#         "task": "l"
#     },
#     "Stimulus_bigHsmallStaskG": {
#         "BIDS_events": "Stimulus/BigLetterh/SmallLetters/Taskg",
#         "bigLetter": "h",
#         "smallLetter": "s",
#         "task": "g"
#     },
#     "Stimulus_bigHsmallStaskL": {
#         "BIDS_events": "Stimulus/BigLetterh/SmallLetters/Taskl",
#         "bigLetter": "h",
#         "smallLetter": "s",
#         "task": "l"
#     }
# }


# # big letter details
# stimulus_big_letter_conditions = {
#     "Stimulus_bigS": {
#         "BIDS_events": "Stimulus/BigLetters",
#         "bigLetter": "s",
#     },
#     "Stimulus_bigH": {
#         "BIDS_events": "Stimulus/BigLetterh",
#         "bigLetter": "h",
#     }
# }

# # small letter details
# stimulus_small_letter_conditions = {
#     "Stimulus_smallS": {
#         "BIDS_events": "Stimulus/SmallLetters",
#         "smallLetter": "s",
#     },
#     "Stimulus_smallH": {
#         "BIDS_events": "Stimulus/SmallLetterh",
#         "smallLetter": "h",
#     }
# }

# # task details
# stimulus_task_conditions = {
#     "Stimulus_taskG": {
#         "BIDS_events": "Stimulus/Taskg",
#         "task": "g",
#     },
#     "Stimulus_taskL": {
#         "BIDS_events": "Stimulus/Taskl",
#         "task": "l",
#     }
# }

# # congruency
# response_congruency_conditions = {
#     "Response_c": {
#         "BIDS_events": ["Response/c25.0", "Response/c75.0"],
#         "congruency": "c"
#     },
#     "Response_i": {
#         "BIDS_events": ["Response/i25.0", "Response/i75.0"],
#         "congruency": "i"
#     }
# }

# # switch type
# response_switch_type_conditions = {
#     "Response_r": {
#         "BIDS_events": ["Response/r25.0", "Response/r75.0"],
#         "switchType": "r"
#     },
#     "Response_s": {        
#         "BIDS_events": ["Response/s25.0", "Response/s75.0"],
#         "switchType": "s"
#     }
# }

# response_experiment_conditions = {
#     "Response_i25s25": {
#         "BIDS_events": "Response/i25.0/s25.0",
#         "congruency": "i",
#         "congruencyProportion": "75%",
#         "switchType": "s",
#         "switchProportion": "25%"
#     },
#     "Response_i25s75": {
#         "BIDS_events": "Response/i25.0/s75.0",
#         "congruency": "i",
#         "congruencyProportion": "75%",
#         "switchType": "s",
#         "switchProportion": "75%"
#     },
#     "Response_i75s25": {
#         "BIDS_events": "Response/i75.0/s25.0",
#         "congruency": "i",
#         "congruencyProportion": "25%",
#         "switchType": "s",
#         "switchProportion": "25%"
#     },
#     "Response_i75s75": {
#         "BIDS_events": "Response/i75.0/s75.0",
#         "congruency": "i",
#         "congruencyProportion": "25%",
#         "switchType": "s",
#         "switchProportion": "75%"
#     },
#     "Response_i25r25": {
#         "BIDS_events": "Response/i25.0/r25.0",
#         "congruency": "i",
#         "congruencyProportion": "75%",
#         "switchType": "r",
#         "switchProportion": "25%"
#     },
#     "Response_i25r75": {
#         "BIDS_events": "Response/i25.0/r75.0",
#         "congruency": "i",
#         "congruencyProportion": "75%",
#         "switchType": "r",
#         "switchProportion": "75%"
#     },
#     "Response_i75r25": {
#         "BIDS_events": "Response/i75.0/r25.0",
#         "congruency": "i",
#         "congruencyProportion": "25%",
#         "switchType": "r",
#         "switchProportion": "25%"
#     },
#     "Response_i75r75": {
#         "BIDS_events": "Response/i75.0/r75.0",
#         "congruency": "i",
#         "congruencyProportion": "25%",
#         "switchType": "r",
#         "switchProportion": "75%"
#     },
#     "Response_c25s25": {
#         "BIDS_events": "Response/c25.0/s25.0",
#         "congruency": "c",
#         "congruencyProportion": "75%",
#         "switchType": "s",
#         "switchProportion": "25%"
#     },
#     "Response_c25s75": {
#         "BIDS_events": "Response/c25.0/s75.0",
#         "congruency": "c",
#         "congruencyProportion": "75%",
#         "switchType": "s",
#         "switchProportion": "75%"
#     },
#     "Response_c75s25": {
#         "BIDS_events": "Response/c75.0/s25.0",
#         "congruency": "c",
#         "congruencyProportion": "25%",
#         "switchType": "s",
#         "switchProportion": "25%"
#     },
#     "Response_c75s75": {
#         "BIDS_events": "Response/c75.0/s75.0",
#         "congruency": "c",
#         "congruencyProportion": "25%",
#         "switchType": "s",
#         "switchProportion": "75%"
#     },
#     "Response_c25r25": {
#         "BIDS_events": "Response/c25.0/r25.0",
#         "congruency": "c",
#         "congruencyProportion": "75%",
#         "switchType": "r",
#         "switchProportion": "25%"
#     },
#     "Response_c25r75": {
#         "BIDS_events": "Response/c25.0/r75.0",
#         "congruency": "c",
#         "congruencyProportion": "75%",
#         "switchType": "r",
#         "switchProportion": "75%"
#     },
#     "Response_c75r25": {
#         "BIDS_events": "Response/c75.0/r25.0",
#         "congruency": "c",
#         "congruencyProportion": "25%",
#         "switchType": "r",
#         "switchProportion": "25%"
#     },
#     "Response_c75r75": {
#         "BIDS_events": "Response/c75.0/r75.0",
#         "congruency": "c",
#         "congruencyProportion": "25%",
#         "switchType": "r",
#         "switchProportion": "75%"
#     }
# }

# response_conditions = {
#     "Response_bigSsmallHtaskG": {
#         "BIDS_events": "Response/BigLetters/SmallLetterh/Taskg",
#         "bigLetter": "s",
#         "smallLetter": "h",
#         "task": "g"
#     },
#     "Response_bigSsmallHtaskL": {
#         "BIDS_events": "Response/BigLetters/SmallLetterh/Taskl",
#         "bigLetter": "s",
#         "smallLetter": "h",
#         "task": "l"
#     },
#     "Response_bigSsmallStaskG": {
#         "BIDS_events": "Response/BigLetters/SmallLetters/Taskg",
#         "bigLetter": "s",
#         "smallLetter": "s",
#         "task": "g"
#     },
#     "Response_bigSsmallStaskL": {
#         "BIDS_events": "Response/BigLetters/SmallLetters/Taskl",
#         "bigLetter": "s",
#         "smallLetter": "s",
#         "task": "l"
#     },
#     "Response_bigHsmallHtaskG": {
#         "BIDS_events": "Response/BigLetterh/SmallLetterh/Taskg",
#         "bigLetter": "h",
#         "smallLetter": "h",
#         "task": "g"
#     },
#     "Response_bigHsmallHtaskL": {
#         "BIDS_events": "Response/BigLetterh/SmallLetterh/Taskl",
#         "bigLetter": "h",
#         "smallLetter": "h",
#         "task": "l"
#     },
#     "Response_bigHsmallStaskG": {
#         "BIDS_events": "Response/BigLetterh/SmallLetters/Taskg",
#         "bigLetter": "h",
#         "smallLetter": "s",
#         "task": "g"
#     },
#     "Response_bigHsmallStaskL": {
#         "BIDS_events": "Response/BigLetterh/SmallLetters/Taskl",
#         "bigLetter": "h",
#         "smallLetter": "s",
#         "task": "l"
#     }
# }


# # big letter details
# response_big_letter_conditions = {
#     "Response_bigS": {
#         "BIDS_events": "Response/BigLetters",
#         "bigLetter": "s",
#     },
#     "Response_bigH": {
#         "BIDS_events": "Response/BigLetterh",
#         "bigLetter": "h",
#     }
# }

# # small letter details
# response_small_letter_conditions = {
#     "Response_smallS": {
#         "BIDS_events": "Response/SmallLetters",
#         "smallLetter": "s",
#     },
#     "Response_smallH": {
#         "BIDS_events": "Response/SmallLetterh",
#         "smallLetter": "h",
#     }
# }

# # task details
# response_task_conditions = {
#     "Response_taskG": {
#         "BIDS_events": "Response/Taskg",
#         "task": "g",
#     },
#     "Response_taskL": {
#         "BIDS_events": "Response/Taskl",
#         "task": "l",
#     }
# }

actually let's just use this small, simple dict for now 2/28

In [ ]:
# Mapping condition names to their corresponding output names and event lists
conditions_and_output_names_and_events = {
    'incongruent': {
        'output_name': 'Stimulus_i25and75_fixationCrossBase_0.5sec',
        'events': ['Stimulus/i75.0', 'Stimulus/i25.0']
    },
    'congruent': {
        'output_name': 'Stimulus_c25and75_fixationCrossBase_0.5sec',
        'events': ['Stimulus/c75.0', 'Stimulus/c25.0']
    },
    'switch': {
        'output_name': 'Stimulus_s25and75_fixationCrossBase_0.5sec',
        'events': ['Stimulus/i75.0/s25.0', 'Stimulus/i25.0/s75.0', 'Stimulus/c75.0/s25.0', 'Stimulus/c25.0/s75.0']
    },
    'repeat': {
        'output_name': 'Stimulus_r25and75_fixationCrossBase_0.5sec',
        'events': ['Stimulus/i75.0/r25.0', 'Stimulus/i25.0/r75.0', 'Stimulus/c75.0/r25.0', 'Stimulus/c25.0/r75.0']
    }
}


make or load wavelets, and get their significant cluster differences

In [3]:
times = [-0.5, 1.5]

if make_wavelets:
    # For making wavelets, extract events directly from the configuration dictionary.
    incongruent_events = conditions_and_output_names_and_events['incongruent']['events']
    congruent_events   = conditions_and_output_names_and_events['congruent']['events']
    switch_events      = conditions_and_output_names_and_events['switch']['events']
    repeat_events      = conditions_and_output_names_and_events['repeat']['events']

    sig_wavelet_differences_per_subject = {}

    for sub in subjects:
        sig_wavelet_differences_per_subject[sub] = {}
        # Preprocess and compute average reaction time (if desired)
        good = get_good_data(sub, layout)
        RTs, skipped = calculate_RTs(good)
        avg_RT = np.median(RTs)
        print(f"Subject {sub} average RT: {avg_RT}")

        # do inc-con, and also switch-repeat
        congruency_mask, congruency_pvals = make_and_get_sig_wavelet_differences(
            sub, layout, incongruent_events, congruent_events, times,
            stat_func=mean_diff, p_thresh=0.05, ignore_adjacency=1, n_perm=100, n_jobs=1)
        switch_type_mask, switch_type_pvals = make_and_get_sig_wavelet_differences(
            sub, layout, switch_events, repeat_events, times,
            stat_func=mean_diff, p_thresh=0.05, ignore_adjacency=1, n_perm=100, n_jobs=1)

        sig_wavelet_differences_per_subject[sub]['congruency'] = (congruency_mask, congruency_pvals)
        sig_wavelet_differences_per_subject[sub]['switch_type'] = (switch_type_mask, switch_type_pvals)

else:
    # For loading precomputed wavelets, use output names from the configuration.
    # Here, 'rescaled' toggles whether to load baseline-corrected (rescaled) TFRs.
    rescaled = False  # Set to True if you wish to use baseline-corrected (dB scaled) wavelets

    sig_wavelet_differences_per_subject = {}

    for sub in subjects:
        sig_wavelet_differences_per_subject[sub] = {}
        good = get_good_data(sub, layout)
        RTs, skipped = calculate_RTs(good)
        avg_RT = np.median(RTs)
        print(f"Subject {sub} average RT: {avg_RT}")

        # do inc-con, and also switch-repeat
        congruency_mask, congruency_pvals = load_and_get_sig_wavelet_differences(
            sub, layout,
            conditions_and_output_names_and_events['incongruent']['output_name'],
            conditions_and_output_names_and_events['congruent']['output_name'],
            rescaled,
            stat_func=mean_diff, p_thresh=0.05, ignore_adjacency=1, n_perm=100, n_jobs=1)
        # switch_type_mask, switch_type_pvals = load_and_get_sig_wavelet_differences(
        #     sub, layout,
        #     conditions_and_output_names_and_events['switch']['output_name'],
        #     conditions_and_output_names_and_events['repeat']['output_name'],
        #     rescaled,
        #     stat_func=mean_diff, p_thresh=0.05, ignore_adjacency=1, n_perm=100, n_jobs=1)

        sig_wavelet_differences_per_subject[sub]['congruency'] = (congruency_mask, congruency_pvals)
        # sig_wavelet_differences_per_subject[sub]['switch_type'] = (switch_type_mask, switch_type_pvals)

Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_ieeg.edf...


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Omitted 228 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\io.py:113: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path, verbose=verbose)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.


...

<RawEDF | sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_ieeg.edf, 178 x 8243200 (4025.0 s), ~201 kB, data not loaded>
outlier round 1 channels: ['RAMT8']
outlier round 2 channels: ['RAMT8', 'RPI16']
outlier round 2 channels: ['RAMT8', 'RPI16', 'LAMT12']
Reading 0 ... 3433906  =      0.000 ...  1676.712 secs...
Applying average reference.
Applying a custom ('sEEG',) reference.
Used Annotations descriptions: ['1', '2', 'Response/c25.0/n25.0/BigLetters/SmallLetters/Taskl/TargetLetters/Responded1.0/ParticipantResponse115.0/CorrectResponse115.0/TrialCount225.0/BlockTrialCount1.0/ReactionTime2149.0/Accuracy1.0/D57', 'Response/c25.0/r25.0/BigLetterh/SmallLetterh/Taskg/TargetLetterh/Responded1.0/ParticipantResponse114.0/CorrectResponse114.0/TrialCount227.0/BlockTrialCount3.0/ReactionTime883.0/Accuracy1.0/D57', 'Response/c25.0/r25.0/BigLetterh/SmallLetterh/Taskg/TargetLetterh/Responded1.0/ParticipantResponse114.0/CorrectResponse114.0/TrialCount228.0/BlockTrialCount4.0/ReactionTime766.0/Ac

c:\Users\jz421\Desktop\GlobalLocal\wavelet_functions.py:63: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  all_trials = mne.concatenate_epochs(all_trials_list)


Not setting metadata
112 matching events found
No baseline correction applied
Data is self data: True


[Parallel(n_jobs=1)]: Done   1 tasks      | elapsed:    2.1s
[Parallel(n_jobs=1)]: Done   4 tasks      | elapsed:    8.9s
[Parallel(n_jobs=1)]: Done   7 tasks      | elapsed:   15.8s
[Parallel(n_jobs=1)]: Done  12 tasks      | elapsed:   27.2s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:   38.8s
[Parallel(n_jobs=1)]: Done  24 tasks      | elapsed:   54.7s
[Parallel(n_jobs=1)]: Done  31 tasks      | elapsed:  1.2min
[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:  1.5min
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:  1.9min
[Parallel(n_jobs=1)]: Done  60 tasks      | elapsed:  2.3min
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:  2.8min
[Parallel(n_jobs=1)]: Done  84 tasks      | elapsed:  3.2min
[Parallel(n_jobs=1)]: Done  97 tasks      | elapsed:  3.7min
[Parallel(n_jobs=1)]: Done 112 tasks      | elapsed:  4.3min
[Parallel(n_jobs=1)]: Done 127 tasks      | elapsed:  4.9min
[Parallel(n_jobs=1)]: Done 144 tasks      | elapsed:  5.5min
[Parallel(n_jobs=1)]: Do

Channels marked as bad:
none
Channels marked as bad:
none
Channels marked as bad:
none
Channels marked as bad:
none
Channels marked as bad:
none


plot time-freq clusters

In [ ]:
for sub in subjects:
    save_dir = os.path.join(layout.root, 'derivatives', 'spec', 'wavelet', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    fig, axs = plt.subplots(6, 10, figsize=(20, 20))
    for i, ax in enumerate(axs.flat):
        if i >= mask.shape[0]:
            ax.axis('off')
            continue
        ax.imshow(mask[i])
        ax.set_title(resp.info['ch_names'][i])

